In [38]:
import sys
import numpy as np
import matplotlib.pyplot as plt
import os
from scipy.interpolate import interp1d
from pathlib import Path
import math

In [41]:
class MassTrack:
    def __init__(data):
        global model

    def get_ZAMS(data):
        global ZAMS_array
        try:
            cut_array = data[data[:,28]> data[1,28] - 0.0015]
            Lfus = np.sum(cut_array[:, 13:17], axis = 1)
            Ltot = (10**(cut_array[:, 3]))* (3.846 * 10**33) # 3.828
            LfusdivLtot = np.divide(Lfus, Ltot)
            ZAMS_age = cut_array[0:LfusdivLtot.argmax(), 2]
            LfusdivLtot_lim = LfusdivLtot[0:LfusdivLtot.argmax()]
            #plt.plot(LfusdivLtot_lim, ZAMS_age)

            ZAMS_x = LfusdivLtot_lim
            ZAMS_y = ZAMS_age

            int_curve = interp1d(ZAMS_x, ZAMS_y)
            interp_age = int_curve(0.999)
            int_curve_two = interp1d(data[:,2], data[:,0:], axis=0)
            ZAMS = np.array(int_curve_two(interp_age))
            return ZAMS

        except:
            #print("followed except")
            central_temperature = cut_array[:,23]
            ZAMS = data[central_temperature.argmax(),:]
            ##ZAMS_array = np.row_stack((ZAMS_array, ZAMS))
            return ZAMS

    def get_IAMS(data):
        global IAMS_array
        try:
            IAMS_x = data[:,28]
            IAMS_y = data[:,2]
            IAMS_int_curve = interp1d(IAMS_x, IAMS_y)
            IAMS_age = IAMS_int_curve(0.3)
            IAMS_int_curve_two = interp1d(data[:,2], data[:,0:], axis=0)
            IAMS = np.array(IAMS_int_curve_two(IAMS_age))
            return IAMS
            ##IAMS_array = np.row_stack((IAMS_array, IAMS))
            ##print("good")

        except:
            #print("followed except IAMS")
            IAMS_error = np.zeros(shape = [1, 61])
            IAMS = data[data[:,2].argmax(), :]
            return IAMS
            #IAMS_array = np.row_stack((IAMS_array, IAMS_error))
            #print("error")

    def get_TAMS(data):
        global TAMS_array
        try:
            TAMS_x = data[:,28]
            TAMS_y = data[:,2]
            TAMS_int_curve = interp1d(TAMS_x, TAMS_y)
            global TAMS_age 
            TAMS_age = TAMS_int_curve(10e-12)
            TAMS_int_curve_two = interp1d(data[:,2], data[:,0:], axis=0)
            TAMS = np.array(TAMS_int_curve_two(TAMS_age))
            return TAMS
        except:
            #print("followed except TAMS")
            Col = data[0,:]
            ## TAMS_age set to 100 so that when TAMS_age is used to filter data in RGBTip_Function, the program errors and doesnt run for models that dont make it past the TAMS.
            TAMS_age = 100
            End = Col.argmax()
            TAMS2 = np.array(data[:,End])
            TAMS = data[data[:,2].argmax(), :]
            return TAMS

    def get_RGBTip(data):
        global RGBTip_array
        try:
            cut_array = data[data[:, 2] > TAMS_age]
            Lum = (10**(cut_array[:, 3]))* (3.846 * 10**33)
            Lum_Max = Lum.argmax()
            Teff = (10**(cut_array[:, 6]))
            Teff_Min = Teff.argmin()
            if (Teff_Min > Lum_Max):
                RGBTip = cut_array[Lum_Max,:]
                return RGBTip
            else:
                RGBTip = cut_array[Teff_Min,:]
                return RGBTip
        except:
            #print("followed except RGB")
            TAMS = data[data[:,2].argmax(), :]
            return TAMS
            RGBTip_error = np.arange(60)
            RGBTip_error = RGBTip_error.fill(0)

    def get_sEEPs(data):
        global sEEPs
        sEEp_ZAMS = (MassTrack.get_ZAMS(data))
        sEEp_IAMS = (MassTrack.get_IAMS(data))
        sEEp_TAMS = (MassTrack.get_TAMS(data))
        sEEp_RGBTip = (MassTrack.get_RGBTip(data))
        
        #weight values for secondary EEP distribution
        Weight_Val_Age = 1.0  #4.0
        Weight_Val_Lum = 8.0  #10.0
        Weight_Val_Temp = 15.0  #1.25
        Weight_Val_T = 0
        

        ZAMS_weight = math.sqrt((Weight_Val_Age*(sEEp_ZAMS[2])**2)+(Weight_Val_Temp*(sEEp_ZAMS[6])**2)+(Weight_Val_Lum*(sEEp_ZAMS[3])**2) +(Weight_Val_T*(sEEp_ZAMS[23])**2))
        '''print(ZAMS_weight)
        IAMS_weight = ZAMS_weight + math.sqrt((Weight_Val_Age*(sEEp_IAMS[2]-sEEp_ZAMS[2])**2)+(Weight_Val_Temp*(sEEp_IAMS[6]-sEEp_ZAMS[6])**2)+(Weight_Val_Lum*(sEEp_IAMS[3]-sEEp_ZAMS[3])**2)+(Weight_Val_T*(sEEp_IAMS[23]-sEEp_ZAMS[23])**2))
        print(IAMS_weight)
        TAMS_weight = IAMS_weight + math.sqrt((Weight_Val_Age*(sEEp_TAMS[2]-sEEp_IAMS[2])**2)+(Weight_Val_Temp*(sEEp_TAMS[6]-sEEp_IAMS[6])**2)+(Weight_Val_Lum*( sEEp_TAMS[3]-sEEp_IAMS[3])**2)+(Weight_Val_T*(sEEp_TAMS[23]-sEEp_IAMS[23])**2))
        print(TAMS_weight)
        RGBTip_weight = TAMS_weight + math.sqrt((Weight_Val_Age*(sEEp_RGBTip[2]-sEEp_TAMS[2])**2)+(Weight_Val_Temp*(sEEp_RGBTip[6]-sEEp_TAMS[6])**2)+(Weight_Val_Lum*(sEEp_RGBTip[3]-sEEp_TAMS[3])**2)+(Weight_Val_T*(sEEp_RGBTip[23]-sEEp_TAMS[23])**2))
        print(RGBTip_weight)'''

        b = data[data[:,2] >= sEEp_ZAMS[2]]
        d = data
        n=0
        point_weight = np.array([ZAMS_weight])
        
        for row in b:
            if(n < b.shape[0]-1):
                ##find the weight of all plotted points in the mass track
                weight = point_weight[n] + math.sqrt((Weight_Val_Age*(b[n+1,2]-b[n,2])**2)+(Weight_Val_Temp*(b[n+1,6]-b[n,6])**2)+(Weight_Val_Lum*(b[n+1,3]-b[n,3])**2)+(Weight_Val_T*(b[n+1,23]-b[n,23])**2))
                point_weight = np.row_stack([point_weight, weight])
                n = n+1
            else:
                n=n

        c = np.column_stack((b,point_weight))
        
        IAMS_weight = MassTrack.get_IAMS(c)[61]
        TAMS_weight = MassTrack.get_TAMS(c)[61]
        RGBTip_weight = MassTrack.get_RGBTip(c)[61]
        
        Dzi = np.linspace(ZAMS_weight, IAMS_weight,202)
        Dtr = np.linspace(TAMS_weight, RGBTip_weight, 202)
        Dit = np.linspace(IAMS_weight, TAMS_weight, 202)
        
#        print(c[:,61])

        D_curve = interp1d(c[:,61], c[:,0:], axis=0)
        
#         plt.plot((D_curve(RGBTip_weight)[2]), D_curve(RGBTip_weight)[3], color='black', marker='o')
#         plt.plot((D_curve(ZAMS_weight)[2]), D_curve(ZAMS_weight)[3], color='blue', marker='o')
#         plt.plot((D_curve(IAMS_weight)[2]), D_curve(IAMS_weight)[3], color='green', marker='o')
#         plt.plot((D_curve(TAMS_weight)[2]), D_curve(TAMS_weight)[3], color='yellow', marker='o')
        
        properties_ZtoI = D_curve(Dzi)
        properties_ItoT = D_curve(Dit)
        properties_TtoR = D_curve(Dtr)
        sEEPs = np.concatenate((properties_ZtoI, properties_ItoT, properties_TtoR), axis=0)
        sEEPs = np.delete(sEEPs, (61), axis=1)
        return sEEPs

In [42]:
#Setting up dictionarys for storing track EEP values
models = {}
global models

ZAMS = "ZAMS"
IAMS = "IAMS"
TAMS = "TAMS"
RGBTip = "RGBTip"
sEEP = "sEEP"


#Change folder to directory of tracks
for (root, dirs, file) in os.walk('./solar_models_test/'):
    for track in file:
        model =('./solar_models_test/' + track)
        data = np.genfromtxt(model, usecols = range(0,61))
        models[track] = {}
        #print(track)
        models[track][ZAMS] = MassTrack.get_ZAMS(data)
        models[track][IAMS] = MassTrack.get_IAMS(data)
        models[track][TAMS] = MassTrack.get_TAMS(data)
        models[track][RGBTip] = MassTrack.get_RGBTip(data)
        models[track][sEEP]= MassTrack.get_sEEPs(data)


[15.91272915 15.95598433 16.00053716 16.04642659 16.0936927  16.14237679
 16.19252142 16.24417038 16.29736881 16.3521632  16.40860141 16.46673278
 16.52660808 16.58827964 16.65180135 16.71722871 16.7846189  16.85403078
 16.92552504 16.99916411 17.07501236 17.15313604 17.23360345 17.31648488
 17.40185274 17.48978164 17.58034842 17.67363219 17.76971447 17.86867924
 17.97061293 18.07560464 18.1837461  18.29513181 18.40985909 18.52802818
 18.64974235 18.77510794 18.9042345  19.03723485 19.17422523 19.3153253
 19.46065839 19.61035146 19.76453533 19.92334471 20.08691837 20.25539925
 20.42893455 20.60767591 20.7917795  20.98140621 21.17672172 21.37789669
 21.58510692 21.79853345 22.01836277 22.24478697 22.47800391 22.71821735
 22.9656372  23.22047964 23.48296735 23.75332968 24.0318029  24.31863032
 24.6140626  24.91835774 25.23178178 25.55460853 25.88712019 26.22960705
 26.58236862 26.94571299 27.31995776 27.70542975 28.10246603 28.51141333
 28.93262903 29.36648123 29.81334904 30.27362275 30.

[15.24704828 15.26828763 15.29016416 15.31269699 15.33590581 15.35981089
 15.38443313 15.40979403 15.43591577 15.46282115 15.4905337  15.51907763
 15.54847787 15.57876012 15.60995084 15.64207727 15.67516752 15.70925045
 15.74435588 15.78051448 15.81775783 15.85611848 15.89562995 15.93632677
 15.97824449 16.02141975 16.06589026 16.11169489 16.15887366 16.20746779
 16.25751975 16.30907327 16.3621734  16.41686653 16.47320045 16.53122438
 16.59098905 16.65254665 16.71595099 16.78125744 16.84852311 16.91780674
 16.98916888 17.06267188 17.13837998 17.21635934 17.29667807 17.37940636
 17.4646165  17.55238296 17.6427824  17.73589384 17.83179862 17.93058055
 18.03232595 18.13712371 18.24506541 18.35624536 18.47076072 18.58871154
 18.7102009  18.83533494 18.96422302 19.09697773 19.23371511 19.37455462
 19.51961932 19.66903597 19.82293513 19.98145128 20.14472293 20.31289274
 20.48610765 20.66451904 20.84828278 21.03755945 21.23251443 21.43331809
 21.64014588 21.85317853 22.07260217 22.29860856 22

[14.83948132 14.85237649 14.86565854 14.87933908 14.89343007 14.90794382
 14.922893   14.93829067 14.9541503  14.97048575 14.98731127 15.00464158
 15.02249182 15.04087758 15.05981494 15.07932045 15.09941114 15.12010458
 15.14141885 15.16337258 15.18598495 15.20927572 15.23326525 15.25797451
 15.28342509 15.30963923 15.33663984 15.36445051 15.39309556 15.42260014
 15.45298987 15.4842913  15.51653179 15.54973957 15.58394368 15.61917401
 15.65546134 15.69283737 15.73133478 15.77098721 15.81182931 15.85389678
 15.89722639 15.941856   15.98782462 16.03517242 16.08394078 16.13417231
 16.18591091 16.23920179 16.29409155 16.35062811 16.40886095 16.46884091
 16.53062038 16.5942534  16.65979554 16.72730413 16.79683812 16.86845821
 16.94222714 17.01820916 17.09647059 17.17707981 17.26010735 17.34562572
 17.43370965 17.52443605 17.61788416 17.71413563 17.81327453 17.91538747
 18.02056363 18.12889473 18.24047558 18.35540378 18.47377955 18.59570619
 18.72129025 18.85064151 18.98387299 19.12110095 19

[14.67864659 14.68636992 14.69427283 14.70237768 14.71071108 14.71928528
 14.72811669 14.73721884 14.74660008 14.75626704 14.76623583 14.77650742
 14.78709251 14.79799952 14.80923966 14.82081933 14.83274509 14.84502723
 14.85767947 14.87070976 14.88412765 14.89794396 14.9121747  14.92683469
 14.94192857 14.95747555 14.97348372 14.98997193 15.00694853 15.02443392
 15.04243691 15.06097982 15.08007123 15.0997359  15.11999035 15.14085319
 15.16233344 15.18445729 15.20724369 15.23070399 15.25486663 15.27974571
 15.3053715  15.33176716 15.35894717 15.38694267 15.41577923 15.44547084
 15.47605583 15.50755871 15.54000702 15.57342733 15.60784765 15.64329068
 15.67979584 15.71739602 15.75612399 15.79600742 15.83708718 15.87939956
 15.92298114 15.96786476 16.01409467 16.06171169 16.11075668 16.16127217
 16.21329891 16.26688597 16.32208056 16.37893036 16.43748485 16.49779474
 16.5599125  16.62389247 16.68979073 16.75766466 16.8275741  16.89958272
 16.97375027 17.05014161 17.12882344 17.20986479 17

[14.53840671 14.5459622  14.55345519 14.56074468 14.56811058 14.57544741
 14.58284289 14.59033082 14.59798615 14.60583456 14.6138719  14.62214305
 14.63065894 14.63943627 14.64847671 14.65779052 14.66739031 14.6772797
 14.68746636 14.69796133 14.70876808 14.71989724 14.73136017 14.7431664
 14.75532329 14.76784486 14.78074056 14.79402064 14.80769664 14.82178302
 14.83628999 14.85122975 14.86661622 14.8824636  14.89878542 14.91559539
 14.93290835 14.95073995 14.96910542 14.98802105 15.00750258 15.02756724
 15.04823296 15.06951841 15.09144165 15.11402178 15.13727817 15.16123119
 15.18590197 15.21131225 15.23748432 15.26444108 15.2922061  15.32080366
 15.35025878 15.38059721 15.4118455  15.44403095 15.47718171 15.51132675
 15.54649583 15.58271983 15.62002989 15.65845937 15.69804142 15.7388108
 15.7808034  15.82405543 15.86860476 15.91449068 15.96175301 16.01043304
 16.06057344 16.11221812 16.16541201 16.22020161 16.27663478 16.33476061
 16.39463049 16.45629611 16.51981191 16.58523283 16.65

[14.32776288 14.34943705 14.37109924 14.3920651  14.41213853 14.43110951
 14.44830496 14.46351051 14.47666329 14.4876716  14.49620083 14.50275466
 14.50738323 14.51058108 14.51295649 14.51498795 14.51713782 14.51957331
 14.52229633 14.52514888 14.52803264 14.53092537 14.53374278 14.53654319
 14.53931653 14.54210156 14.54489722 14.54773988 14.55066217 14.55366782
 14.55676001 14.55994384 14.56322314 14.56660052 14.57008072 14.57366894
 14.57735914 14.58116067 14.5850747  14.58910584 14.59325813 14.59753433
 14.60194101 14.60647978 14.61115324 14.61596683 14.62092312 14.62602871
 14.63128946 14.63670549 14.64228354 14.64802892 14.65394652 14.66004166
 14.66631955 14.67278634 14.67944666 14.68630699 14.69337364 14.70065111
 14.70814759 14.71586895 14.72382149 14.73201455 14.7404523  14.74914126
 14.75809148 14.76731149 14.77680842 14.7865904  14.79666301 14.80703742
 14.8177247  14.82873265 14.84006928 14.85174669 14.86377417 14.87616375
 14.88892323 14.90206795 14.91560629 14.92954887 14

[14.43806846 14.43911073 14.44007071 ... 34.82541402 34.83367486
 34.84164162]
[14.45436997 14.45548865 14.45656561 ... 41.97513111 41.97532467
 41.97552122]
[14.4714215  14.47316156 14.47455199 ... 40.57665524 40.57694477
 40.57723902]
[14.51365104 14.51747909 14.52067016 ... 38.00621109 38.00669209
 38.00730774]
[14.55530729 14.56158095 14.56726112 ... 35.47674546 35.47674992
 35.47679727]
[14.5958766  14.6051805  14.61337598 ... 33.46939428 33.46940025
 33.46945529]
[14.63641179 14.648216   14.65891217 ... 31.89100014 31.89113428
 31.89259758]
[14.67772873 14.68832497 14.69829763 ... 30.62732925 30.62786356
 30.62869984]
[14.72014917 14.73006718 14.73964733 ... 29.64294908 29.64466219
 29.65115242]
[14.76264289 14.77359552 14.78412394 ... 28.84905788 28.85142429
 28.85322168]
[14.80562748 14.81796219 14.83007584 ... 28.21313695 28.21333746
 28.2135393 ]
[14.84944345 14.86322614 14.8769245  14.89005456 14.90271967 14.91473797
 14.92600462 14.93638359 14.94579154 14.95407573 14.961213